In [1]:
import factory

class StudentFactory(factory.Factory):
    class Meta:
        model = dict
    id = factory.Sequence(lambda n: 'stu{}'.format(n))
    tid = factory.Sequence(lambda n: 'Tstu{}'.format(n))

StudentFactory()

{'id': 'stu0', 'tid': 'Tstu0'}

In [2]:
StudentFactory()

{'id': 'stu1', 'tid': 'Tstu1'}

In [3]:
class KeyFactoryField:
    def __init__(self, keyfactory, field):
        self.keyfactory = keyfactory
        self.field = field

    def __getitem__(self, key=None):
        instance = self.keyfactory.instance(key)
        if self.field in instance:
            return instance[self.field]
        else:
            raise KeyFactoryFieldError('Key field "{}" not defined for factory {}'.format(self.field, self.keyfactory.factory))

class KeyFactory:
    def __init__(self, factory):
        self.factory = factory
        self.case = None
        self.cached = {}
        self.next_case(None)

    def next_case(self, case):
        self.case = case
        self.cached[self.case] = {}

    def __getitem__(self, field):
        return KeyFactoryField(self, field)

    def instance(self, key=None):
        key = os.urandom(32) if key is None else key
        if key not in self.cached[self.case]:
            self.cached[self.case][key] = self.factory()

        return self.cached[self.case][key]

    def case_lookup(self, field):
        lkp = {}
        for case_id, cached_keys in self.cached.items():
            for keys in cached_keys.values():
                lkp[keys['id']] = case_id
        return lkp
        
kf = KeyFactory(StudentFactory)
    

In [4]:
for c in range(3):
    kf.next_case(c)
    for i in range(2):
        print(kf['id'][i])

stu2
stu3
stu4
stu5
stu6
stu7


In [7]:
kf.case_lookup('id')['stu6']

2

In [111]:
kf.cached

{139769633356048: {},
 139769633660032: {0: {'id': 'stu13', 'tid': 'Tstu13'},
  1: {'id': 'stu14', 'tid': 'Tstu14'}},
 139769633660064: {0: {'id': 'stu15', 'tid': 'Tstu15'},
  1: {'id': 'stu16', 'tid': 'Tstu16'}},
 139769633660096: {0: {'id': 'stu17', 'tid': 'Tstu17'},
  1: {'id': 'stu18', 'tid': 'Tstu18'}}}

In [114]:
inverted = {}
for case_id, cached_keys in kf.cached.items():
    for keys in cached_keys.values():
        print(c, keys)
        inverted[keys['id']] = case_id
inverted

139769633660096 {'id': 'stu13', 'tid': 'Tstu13'}
139769633660096 {'id': 'stu14', 'tid': 'Tstu14'}
139769633660096 {'id': 'stu15', 'tid': 'Tstu15'}
139769633660096 {'id': 'stu16', 'tid': 'Tstu16'}
139769633660096 {'id': 'stu17', 'tid': 'Tstu17'}
139769633660096 {'id': 'stu18', 'tid': 'Tstu18'}


{'stu13': 139769633660032,
 'stu14': 139769633660032,
 'stu15': 139769633660064,
 'stu16': 139769633660064,
 'stu17': 139769633660096,
 'stu18': 139769633660096}

In [87]:
kf['id'][1]

'stu20'

In [86]:
kf.next_case(1)

In [88]:
kf['fid'][1]

KeyFactoryFieldError: Key field "fid" not defined for factory <StudentFactory for <class 'dict'>>

In [125]:
class Blerbo:
    def __init__(self, arg):
        self.arg = arg
        
b = Blerbo('hi')
d = {}
d[b] = 'hola'
b.arg = 'smerf'
d[b]
b